<a href="https://colab.research.google.com/github/myrah/AAI2025/blob/main/Agent/AgenticAIAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Install prerequisite packages
!pip install python-dotenv
!pip install llama-index-agent-openai
!pip install llama-index
!pip install llama-index-llms-openai
!pip install llama-index-embeddings-openai



  Using cached llama_index_core-0.12.52.post1-py3-none-any.whl.metadata (2.5 kB)
  Using cached llama_index_llms_openai-0.4.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached llama_index_workflows-1.3.0-py3-none-any.whl.metadata (6.4 kB)
Using cached llama_index_core-0.12.52.post1-py3-none-any.whl (7.6 MB)
Using cached llama_index_llms_openai-0.4.7-py3-none-any.whl (25 kB)
Using cached llama_index_workflows-1.3.0-py3-none-any.whl (42 kB)
  Attempting uninstall: llama-index-workflows
    Found existing installation: llama-index-workflows 2.6.0
    Uninstalling llama-index-workflows-2.6.0:
      Successfully uninstalled llama-index-workflows-2.6.0
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.14.4
    Uninstalling llama-index-core-0.14.4:
      Successfully uninstalled llama-index-core-0.14.4
  Attempting uninstall: llama-index-llms-openai
    Found existing installation: llama-index-llms-openai 0.6.3
    Uninstalling llama-index-llms-opena

In [4]:
# Setup OpenAI connection
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
import os
import nest_asyncio
# from openai import OpenAI # Removed conflicting import
from google.colab import userdata

nest_asyncio.apply()

# Set your OpenAI API key (from https://platform.openai.com/api-keys)
api_key=userdata.get("OPENAI_EDU_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key  # optional but recommended

# Configure LlamaIndex to use OpenAI models
Settings.llm = OpenAI()
Settings.llm.model = "gpt-4-turbo"  # Using a more capable model for the router


Settings.embed_model = OpenAIEmbedding(api_key=api_key, model="text-embedding-3-small")  # Pass api_key explicitly

In [5]:
from typing import List
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import  VectorStoreIndex
from llama_index.core.tools import QueryEngineTool

#-------------------------------------------------------------
# Tool 1 : Function that returns the list of items in an order
#-------------------------------------------------------------
def get_order_items(order_id: int) -> List[str] :
    """Given an order Id, this function returns the
    list of items purchased for that order"""

    order_items = {
            1001: ["Laptop","Mouse"],
            1002: ["Keyboard","HDMI Cable"],
            1003: ["Laptop","Keyboard"]
        }
    if order_id in order_items.keys():
        return order_items[order_id]
    else:
        return []

#-------------------------------------------------------------
# Tool 2 : Function that returns the delivery date for an order
#-------------------------------------------------------------
def get_delivery_date(order_id: int) -> str:
    """Given an order Id, this function returns the
    delivery date for that order"""

    delivery_dates = {
            1001: "10-Jun",
            1002: "12-Jun",
            1003: "08-Jun"
    }
    if order_id in delivery_dates.keys():
        return delivery_dates[order_id]
    else:
        return []

#----------------------------------------------------------------
# Tool 3 : Function that returns maximum return days for an item
#----------------------------------------------------------------
def get_item_return_days(item: str) -> int :
    """Given an Item, this function returns the return support
    for that order. The return support is in number of days"""

    item_returns = {
            "Laptop"     : 30,
            "Mouse"      : 15,
            "Keyboard"   : 15,
            "HDMI Cable" : 5
    }
    if item in item_returns.keys():
        return item_returns[item]
    else:
        #Default
        return 45

#-------------------------------------------------------------
# Tool 4 : Vector DB that contains customer support contacts
#-------------------------------------------------------------
#Setup vector index for return policies
support_docs=SimpleDirectoryReader(input_files=["Customer Service.pdf"]).load_data()

splitter=SentenceSplitter(chunk_size=1024)
support_nodes=splitter.get_nodes_from_documents(support_docs)
support_index=VectorStoreIndex(support_nodes)
support_query_engine = support_index.as_query_engine()

In [6]:
from llama_index.core.tools import FunctionTool

#Create tools for the 3 functions and 1 index
order_item_tool = FunctionTool.from_defaults(fn=get_order_items)
delivery_date_tool = FunctionTool.from_defaults(fn=get_delivery_date)
return_policy_tool = FunctionTool.from_defaults(fn=get_item_return_days)

support_tool = QueryEngineTool.from_defaults(
    query_engine=support_query_engine,
    description=(
        "Customer support policies and contact information"
    ),
)

In [7]:
!pip install llama-index-agent-openai
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner
from llama_index.llms.openai import OpenAI

#Setup the Agent worker in LlamaIndex with all the Tools
#This is the tool executor process
agent_worker = FunctionCallingAgentWorker.from_tools(
    [order_item_tool,
     delivery_date_tool,
     return_policy_tool,
     support_tool
    ],
    llm=Settings.llm,
    verbose=True
)
#Create an Agent Orchestrator with LlamaIndex
agent = AgentRunner(agent_worker)


/tmp/ipython-input-2029546619.py:18: DeprecationWarning: Call to deprecated class AgentRunner. (AgentRunner has been deprecated and is not maintained.

This implementation will be removed in a v0.13.0.

See the docs for more information on updated agent usage: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  agent = AgentRunner(agent_worker)


In [8]:
#Get return policy for an order
response = agent.query(
    "What is the return policy for order number 1001"
)

print("\n Final output : \n", response)

Added user message to memory: What is the return policy for order number 1001
=== Calling Function ===
Calling function: get_order_items with args: {"order_id": 1001}
=== Function Output ===
['Laptop', 'Mouse']
=== Calling Function ===
Calling function: get_delivery_date with args: {"order_id": 1001}
=== Function Output ===
10-Jun
=== Calling Function ===
Calling function: get_item_return_days with args: {"item": "Laptop"}
=== Function Output ===
30
=== Calling Function ===
Calling function: get_item_return_days with args: {"item": "Mouse"}
=== Function Output ===
15
=== LLM Response ===
For order number 1001, the return policies for the items are as follows:

- **Laptop**: Can be returned within 30 days of the delivery date.
- **Mouse**: Can be returned within 15 days of the delivery date.

The delivery date for this order was on June 10th.

 Final output : 
 For order number 1001, the return policies for the items are as follows:

- **Laptop**: Can be returned within 30 days of the d

In [9]:
# Get the delivery date for order 1002
response = agent.query(
    "What is the delivery date for order number 1002?"
)

print("\n Final output : \n", response)

Added user message to memory: What is the delivery date for order number 1002?
=== Calling Function ===
Calling function: get_delivery_date with args: {"order_id": 1002}
=== Function Output ===
12-Jun
=== LLM Response ===
The delivery date for order number 1002 is June 12.

 Final output : 
 The delivery date for order number 1002 is June 12.


In [10]:
#Question about an invalid order number
response = agent.query(
    "What is the return policy for order number 1004"
)

print("\n Final output : \n", response)

Added user message to memory: What is the return policy for order number 1004
=== Calling Function ===
Calling function: get_order_items with args: {"order_id": 1004}
=== Function Output ===

=== Calling Function ===
Calling function: get_order_items with args: {"order_id": 1004}
=== Function Output ===

=== Calling Function ===
Calling function: get_order_items with args: {"order_id": 1004}
=== Function Output ===

=== Calling Function ===
Calling function: get_order_items with args: {"order_id": 1004}
=== Function Output ===

=== Calling Function ===
Calling function: get_order_items with args: {"order_id": 1004}
=== Function Output ===


 Final output : 
 None
